In [78]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False
import gc

# 인코더 추가
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
#상수항추가
from statsmodels.tools.tools import add_constant
# 카이제곱, ANOVA
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
#Turkeyhsd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

### 데이터 불러오기

In [4]:
# parquet 파일 데이터를 읽어온다.
# x 값으로 쓸 데이터 
df1_train = pd.read_parquet('open/train/8.성과정보/201807_train_성과정보.parquet')
df2_train = pd.read_parquet('open/train/8.성과정보/201808_train_성과정보.parquet')
df3_train = pd.read_parquet('open/train/8.성과정보/201809_train_성과정보.parquet')
df4_train = pd.read_parquet('open/train/8.성과정보/201810_train_성과정보.parquet')
df5_train = pd.read_parquet('open/train/8.성과정보/201811_train_성과정보.parquet')
df6_train = pd.read_parquet('open/train/8.성과정보/201812_train_성과정보.parquet')

combined_df = pd.concat([df1_train, df2_train, df3_train, df4_train, df5_train, df6_train], axis=0, ignore_index=True)
combined_df

,기준년월,ID,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,증감율_이용건수_카드론_전월,증감율_이용금액_신용_전월,...,변동률_RV일시불평잔,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M
0,201807,TRAIN_000000,0.454548,0.454548,0.364727,1.999996,0.0,0.000000,0.0,0.409766,...,0.999998,1.042805,0.999700,0.999998,0.999998,0.261886,0.270752,0.000000,1.044401,1.280543
1,201807,TRAIN_000001,-0.511808,-0.511808,-0.511808,0.000000,0.0,0.000000,0.0,-1.010817,...,1.092698,0.905663,0.999998,0.999998,0.999998,-0.563388,-0.670348,0.000000,0.000000,0.000000
2,201807,TRAIN_000002,-0.062716,-0.062716,0.067843,-1.999996,0.0,0.000000,0.0,-0.902166,...,1.006124,1.993590,0.852567,0.999998,0.999998,-0.046516,0.058114,-0.014191,0.524159,1.208420
3,201807,TRAIN_000003,0.135707,0.135707,0.153975,-1.999996,0.0,0.000000,0.0,-0.866052,...,0.999998,1.050646,0.999877,0.999998,0.999998,0.023821,0.258943,0.000000,0.880925,1.657124
4,201807,TRAIN_000004,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000028,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,0.000000,0.000000,0.000000,0.000000,0.0,-0.081943,0.0,0.000027,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,NaN,NaN
2399996,201812,TRAIN_399996,0.073804,0.073804,0.073804,0.000000,0.0,0.000000,0.0,0.207099,...,0.999998,0.999998,0.999998,0.999998,0.921733,-0.203251,-0.159143,0.000000,1.377071,2.533815
2399997,201812,TRAIN_399997,-0.040432,-0.040432,-0.040432,0.000000,0.0,0.000000,0.0,0.254495,...,0.999998,0.345027,0.999998,0.999998,0.999998,0.027319,0.126581,0.000000,0.000000,0.000000
2399998,201812,TRAIN_399998,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000034,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,NaN,NaN


In [5]:
# 결과데이터를 불러온다. 
segment_df = pd.read_csv('Segment.csv')
segment_df

,기준년월,ID,Segment
0,201807,TRAIN_000000,D
1,201807,TRAIN_000001,E
2,201807,TRAIN_000002,C
3,201807,TRAIN_000003,D
4,201807,TRAIN_000004,E
...,...,...,...
2399995,201812,TRAIN_399995,E
2399996,201812,TRAIN_399996,D
2399997,201812,TRAIN_399997,C
2399998,201812,TRAIN_399998,E


In [6]:
# 둘을 합쳐준다. 
# ID는 중복되는 값이 concat의 영향으로 6개씩 존재하므로
# 기준년월을 이용해 추가적인 기준을 만들어준다. 
all_df = combined_df.merge(segment_df, on=['ID', '기준년월'], how='left')
all_df

,기준년월,ID,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,증감율_이용건수_카드론_전월,증감율_이용금액_신용_전월,...,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M,Segment
0,201807,TRAIN_000000,0.454548,0.454548,0.364727,1.999996,0.0,0.000000,0.0,0.409766,...,1.042805,0.999700,0.999998,0.999998,0.261886,0.270752,0.000000,1.044401,1.280543,D
1,201807,TRAIN_000001,-0.511808,-0.511808,-0.511808,0.000000,0.0,0.000000,0.0,-1.010817,...,0.905663,0.999998,0.999998,0.999998,-0.563388,-0.670348,0.000000,0.000000,0.000000,E
2,201807,TRAIN_000002,-0.062716,-0.062716,0.067843,-1.999996,0.0,0.000000,0.0,-0.902166,...,1.993590,0.852567,0.999998,0.999998,-0.046516,0.058114,-0.014191,0.524159,1.208420,C
3,201807,TRAIN_000003,0.135707,0.135707,0.153975,-1.999996,0.0,0.000000,0.0,-0.866052,...,1.050646,0.999877,0.999998,0.999998,0.023821,0.258943,0.000000,0.880925,1.657124,D
4,201807,TRAIN_000004,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000028,...,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,NaN,NaN,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,0.000000,0.000000,0.000000,0.000000,0.0,-0.081943,0.0,0.000027,...,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,NaN,NaN,E
2399996,201812,TRAIN_399996,0.073804,0.073804,0.073804,0.000000,0.0,0.000000,0.0,0.207099,...,0.999998,0.999998,0.999998,0.921733,-0.203251,-0.159143,0.000000,1.377071,2.533815,D
2399997,201812,TRAIN_399997,-0.040432,-0.040432,-0.040432,0.000000,0.0,0.000000,0.0,0.254495,...,0.345027,0.999998,0.999998,0.999998,0.027319,0.126581,0.000000,0.000000,0.000000,C
2399998,201812,TRAIN_399998,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000034,...,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,NaN,NaN,E


In [76]:
# 컬럼들을 리스트에 담아 확인한다
column_list = all_df.columns.tolist()
column_list

['증감율_이용건수_신용_전월',
 '증감율_이용건수_신판_전월',
 '증감율_이용건수_일시불_전월',
 '증감율_이용건수_할부_전월',
 '증감율_이용건수_CA_전월',
 '증감율_이용건수_체크_전월',
 '증감율_이용건수_카드론_전월',
 '증감율_이용금액_신용_전월',
 '증감율_이용금액_신판_전월',
 '증감율_이용금액_일시불_전월',
 '증감율_이용금액_할부_전월',
 '증감율_이용금액_CA_전월',
 '증감율_이용금액_체크_전월',
 '증감율_이용금액_카드론_전월',
 '증감율_이용건수_신용_분기',
 '증감율_이용건수_신판_분기',
 '증감율_이용건수_일시불_분기',
 '증감율_이용건수_할부_분기',
 '증감율_이용건수_CA_분기',
 '증감율_이용건수_체크_분기',
 '증감율_이용건수_카드론_분기',
 '증감율_이용금액_신용_분기',
 '증감율_이용금액_신판_분기',
 '증감율_이용금액_일시불_분기',
 '증감율_이용금액_할부_분기',
 '증감율_이용금액_CA_분기',
 '증감율_이용금액_체크_분기',
 '증감율_이용금액_카드론_분기',
 '잔액_신판평균한도소진율_r6m',
 '잔액_신판최대한도소진율_r6m',
 '잔액_신판평균한도소진율_r3m',
 '잔액_신판최대한도소진율_r3m',
 '잔액_신판ca평균한도소진율_r6m',
 '잔액_신판ca최대한도소진율_r6m',
 '잔액_신판ca평균한도소진율_r3m',
 '잔액_신판ca최대한도소진율_r3m',
 '변동률_일시불평잔',
 '변동률_RV일시불평잔',
 '변동률_할부평잔',
 '변동률_CA평잔',
 '변동률_RVCA평잔',
 '변동률_카드론평잔',
 '변동률_잔액_B1M',
 '변동률_잔액_일시불_B1M',
 '변동률_잔액_CA_B1M',
 '혜택수혜율_R3M',
 '혜택수혜율_B0M',
 'Segment',
 'Segment_e']

In [64]:
numerical_cols = ['증감율_이용건수_신용_전월',
 '증감율_이용건수_신판_전월',
 '증감율_이용건수_일시불_전월',
 '증감율_이용건수_할부_전월',
 '증감율_이용건수_CA_전월',
 '증감율_이용건수_체크_전월',

 '증감율_이용금액_신용_전월',
 '증감율_이용금액_신판_전월',
 '증감율_이용금액_일시불_전월',
 '증감율_이용금액_할부_전월',
 '증감율_이용금액_CA_전월',
 '증감율_이용금액_체크_전월',
 
 '증감율_이용건수_신용_분기',
 '증감율_이용건수_신판_분기',
 '증감율_이용건수_일시불_분기',
 '증감율_이용건수_할부_분기',
 '증감율_이용건수_CA_분기',
 '증감율_이용건수_체크_분기',

 '증감율_이용금액_신용_분기',
 '증감율_이용금액_신판_분기',
 '증감율_이용금액_일시불_분기',
 '증감율_이용금액_할부_분기',
 '증감율_이용금액_CA_분기',
 '증감율_이용금액_체크_분기',
 
 '잔액_신판평균한도소진율_r6m',
 '잔액_신판최대한도소진율_r6m',
 '잔액_신판평균한도소진율_r3m',
 '잔액_신판최대한도소진율_r3m',
 '잔액_신판ca평균한도소진율_r6m',
 '잔액_신판ca최대한도소진율_r6m',
 '잔액_신판ca평균한도소진율_r3m',
 '잔액_신판ca최대한도소진율_r3m',
 '변동률_일시불평잔',
 '변동률_RV일시불평잔',
 '변동률_할부평잔',
 '변동률_CA평잔',
 '변동률_RVCA평잔',
 '변동률_카드론평잔',
 '변동률_잔액_B1M',
 '변동률_잔액_일시불_B1M',
 '변동률_잔액_CA_B1M',
 '혜택수혜율_R3M',
 '혜택수혜율_B0M']

categorical_cols = ['증감율_이용건수_카드론_전월', '증감율_이용금액_카드론_전월', '증감율_이용건수_카드론_분기', '증감율_이용금액_카드론_분기']

### 변수간 관계파악
Anova -> ETA제곱
카이제곱 -> 크레이머스V

In [60]:
# 수치형/범주형
num_cols = numerical_cols
cat_cols = categorical_cols

anova = []
chi = []

for col in all_df.columns:
    if col == 'Segment':
        continue

    if col in num_cols:
        data = all_df[[col, 'Segment']].dropna()
        groups = [data[data['Segment'] == val][col] for val in data['Segment'].unique()]
        try:
            stat = f_oneway(*groups).statistic
            ss_between = sum([(g.mean() - data[col].mean())**2 * len(g) for g in groups])
            ss_total = sum((data[col] - data[col].mean())**2)
            eta2 = eta_squared(ss_between, ss_total)
            anova.append({'변수': col, '유형': '수치형', '계수종류': 'Eta²', '상관계수': eta2})
        except:
            continue

    elif col in cat_cols:
        contingency = pd.crosstab(all_df[col], all_df['Segment'])
        if contingency.shape[0] > 1 and contingency.shape[1] > 1:
            try:
                v = cramers_v(contingency)
                chi.append({'변수': col, '유형': '범주형', '계수종류': "Cramér's V", '상관계수': v})
            except:
                continue

# 결과 정리
result_df1 = pd.DataFrame(anova)
result_df2 = pd.DataFrame(chi)
result_df1 = result_df1.sort_values(by='상관계수', ascending=False).reset_index(drop=True)
result_df2 = result_df2.sort_values(by='상관계수', ascending=False).reset_index(drop=True)

# 결과 출력
display(result_df1)
display(result_df2)

,변수,유형,계수종류,상관계수
0,잔액_신판ca최대한도소진율_r6m,수치형,Eta²,0.075072
1,잔액_신판ca최대한도소진율_r3m,수치형,Eta²,0.072253
2,잔액_신판ca평균한도소진율_r3m,수치형,Eta²,0.069510
3,잔액_신판ca평균한도소진율_r6m,수치형,Eta²,0.066496
4,잔액_신판최대한도소진율_r6m,수치형,Eta²,0.044228
5,잔액_신판최대한도소진율_r3m,수치형,Eta²,0.042057
6,잔액_신판평균한도소진율_r3m,수치형,Eta²,0.040209
7,잔액_신판평균한도소진율_r6m,수치형,Eta²,0.037250
8,변동률_RV일시불평잔,수치형,Eta²,0.012765
9,변동률_RVCA평잔,수치형,Eta²,0.012526


,변수,유형,계수종류,상관계수
0,증감율_이용금액_카드론_분기,범주형,Cramér's V,0.024366
1,증감율_이용건수_카드론_분기,범주형,Cramér's V,0.023935
2,증감율_이용금액_카드론_전월,범주형,Cramér's V,0.006537
3,증감율_이용건수_카드론_전월,범주형,Cramér's V,0.005659


In [72]:
pd.to_excel

,변수,유형,계수종류,상관계수
0,증감율_이용금액_카드론_분기,범주형,Cramér's V,0.024366
1,증감율_이용건수_카드론_분기,범주형,Cramér's V,0.023935
2,증감율_이용금액_카드론_전월,범주형,Cramér's V,0.006537
3,증감율_이용건수_카드론_전월,범주형,Cramér's V,0.005659


### 히트맵을 그려본다. 

In [98]:
all_df['증감율_이용금액_카드론_분기'].value_counts()

증감율_이용금액_카드론_분기
 0.000000    2343478
-1.999996      46934
 1.999996       9588
Name: count, dtype: int64

In [104]:
### 인코딩을 진행한다.  
segment_order = {'E': 0, 'D': 1, 'C': 2, 'B': 3, 'A': 4}
all_df['Segment_e'] = all_df['Segment'].map(segment_order)

# 카테고리 매핑 정의
mapping = {
    -1.999996: 0,   # 하락
     0.000000: 1,   # 보합
     1.999996: 2    # 상승
}

# 적용
cols = ['증감율_이용건수_카드론_전월', '증감율_이용금액_카드론_전월', '증감율_이용건수_카드론_분기', '증감율_이용금액_카드론_분기']
# 반복 적용
for col in cols:
    all_df[col] = all_df[col].map(mapping)

In [106]:
all_df['증감율_이용금액_카드론_분기'].value_counts()

증감율_이용금액_카드론_분기
1    2343478
0      46934
2       9588
Name: count, dtype: int64

In [135]:
# row와 columns의 수가 너무 많아서 메모리가 터지니 
# 리스트를 나눠준다. 

column1 = ['증감율_이용건수_신용_전월', '증감율_이용건수_신판_전월', '증감율_이용건수_일시불_전월',
       '증감율_이용건수_할부_전월', '증감율_이용건수_CA_전월', '증감율_이용건수_체크_전월']
          
column2 = ['증감율_이용금액_신용_전월', '증감율_이용금액_신판_전월', '증감율_이용금액_일시불_전월', '증감율_이용금액_할부_전월',
       '증감율_이용금액_CA_전월', '증감율_이용금액_체크_전월']

column3 = ['증감율_이용건수_신용_분기',
       '증감율_이용건수_신판_분기', '증감율_이용건수_일시불_분기', '증감율_이용건수_할부_분기', '증감율_이용건수_CA_분기',
       '증감율_이용건수_체크_분기']

column4 = ['증감율_이용금액_신용_분기', '증감율_이용금액_신판_분기',
       '증감율_이용금액_일시불_분기', '증감율_이용금액_할부_분기', '증감율_이용금액_CA_분기', '증감율_이용금액_체크_분기']

column5 = ['잔액_신판평균한도소진율_r6m', '잔액_신판최대한도소진율_r6m',
       '잔액_신판평균한도소진율_r3m', '잔액_신판최대한도소진율_r3m', '잔액_신판ca평균한도소진율_r6m',
       '잔액_신판ca최대한도소진율_r6m', '잔액_신판ca평균한도소진율_r3m', '잔액_신판ca최대한도소진율_r3m']

column6 = ['변동률_일시불평잔', '변동률_RV일시불평잔', '변동률_할부평잔', '변동률_CA평잔', '변동률_RVCA평잔',
       '변동률_카드론평잔']
 

column7 = ['변동률_잔액_B1M', '변동률_잔액_일시불_B1M', '변동률_잔액_CA_B1M',
       '혜택수혜율_R3M', '혜택수혜율_B0M']
# 범주형 컬럼 
column8 = ['증감율_이용건수_카드론_전월', '증감율_이용금액_카드론_전월', '증감율_이용건수_카드론_분기', '증감율_이용금액_카드론_분기']

In [137]:
print(all_df['Segment_e'].unique())

[1 0 2 4 3]


In [109]:
all_df[column1].corr()

,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,Segment_e
증감율_이용건수_신용_전월,1.000000,0.988043,0.946005,0.147706,0.049353,-0.002850,0.004801
증감율_이용건수_신판_전월,0.988043,1.000000,0.955770,0.149920,0.008959,-0.002809,0.005583
증감율_이용건수_일시불_전월,0.946005,0.955770,1.000000,0.088149,0.008447,-0.002224,0.008263
증감율_이용건수_할부_전월,0.147706,0.149920,0.088149,1.000000,0.040903,0.020261,-0.080531
증감율_이용건수_CA_전월,0.049353,0.008959,0.008447,0.040903,1.000000,0.017631,-0.056228
증감율_이용건수_체크_전월,-0.002850,-0.002809,-0.002224,0.020261,0.017631,1.000000,-0.066659
Segment_e,0.004801,0.005583,0.008263,-0.080531,-0.056228,-0.066659,1.000000


In [139]:
# 비교할 종속변수
target = 'Segment_e'
# 결과를 담을 리스트 
corr_result = []
# 반복
for col in column1:
    if col in all_df.columns:
        corr = all_df[col].corr(all_df[target])
        corr_result.append({'변수': col, '상관계수': corr})

# 데이터프레임으로 정리한다 
result_df3 = pd.DataFrame(corr_result).sort_values(by='상관계수', key=abs, ascending=False).reset_index(drop=True)

display(result_df3)

,변수,상관계수
0,증감율_이용건수_할부_전월,-0.080531
1,증감율_이용건수_체크_전월,-0.066659
2,증감율_이용건수_CA_전월,-0.056228
3,증감율_이용건수_일시불_전월,0.008263
4,증감율_이용건수_신판_전월,0.005583
5,증감율_이용건수_신용_전월,0.004801


In [141]:
# 비교할 종속변수
target = 'Segment_e'
# 결과를 담을 리스트 
corr_result = []
# 반복
for col in column2:
    if col in all_df.columns:
        corr = all_df[col].corr(all_df[target])
        corr_result.append({'변수': col, '상관계수': corr})

# 데이터프레임으로 정리한다 
result_df4 = pd.DataFrame(corr_result).sort_values(by='상관계수', key=abs, ascending=False).reset_index(drop=True)

display(result_df4)

,변수,상관계수
0,증감율_이용금액_일시불_전월,0.073526
1,증감율_이용금액_신판_전월,0.073436
2,증감율_이용금액_신용_전월,0.046525
3,증감율_이용금액_체크_전월,-0.027065
4,증감율_이용금액_할부_전월,-0.021696
5,증감율_이용금액_CA_전월,-0.013888


In [142]:
# 비교할 종속변수
target = 'Segment_e'
# 결과를 담을 리스트 
corr_result = []
# 반복
for col in column3:
    if col in all_df.columns:
        corr = all_df[col].corr(all_df[target])
        corr_result.append({'변수': col, '상관계수': corr})

# 데이터프레임으로 정리한다 
result_df5 = pd.DataFrame(corr_result).sort_values(by='상관계수', key=abs, ascending=False).reset_index(drop=True)

display(result_df5)

,변수,상관계수
0,증감율_이용건수_할부_분기,-0.036098
1,증감율_이용건수_CA_분기,-0.015660
2,증감율_이용건수_체크_분기,-0.009855
3,증감율_이용건수_일시불_분기,-0.001071
4,증감율_이용건수_신용_분기,0.000929
5,증감율_이용건수_신판_분기,0.000335


In [143]:
# 비교할 종속변수
target = 'Segment_e'
# 결과를 담을 리스트 
corr_result = []
# 반복
for col in column4:
    if col in all_df.columns:
        corr = all_df[col].corr(all_df[target])
        corr_result.append({'변수': col, '상관계수': corr})

# 데이터프레임으로 정리한다 
result_df6 = pd.DataFrame(corr_result).sort_values(by='상관계수', key=abs, ascending=False).reset_index(drop=True)

display(result_df6)

,변수,상관계수
0,증감율_이용금액_CA_분기,-0.031328
1,증감율_이용금액_할부_분기,-0.028412
2,증감율_이용금액_신용_분기,-0.014716
3,증감율_이용금액_신판_분기,-0.013519
4,증감율_이용금액_체크_분기,-0.010017
5,증감율_이용금액_일시불_분기,-0.006105


In [144]:
# 비교할 종속변수
target = 'Segment_e'
# 결과를 담을 리스트 
corr_result = []
# 반복
for col in column5:
    if col in all_df.columns:
        corr = all_df[col].corr(all_df[target])
        corr_result.append({'변수': col, '상관계수': corr})

# 데이터프레임으로 정리한다 
result_df7 = pd.DataFrame(corr_result).sort_values(by='상관계수', key=abs, ascending=False).reset_index(drop=True)

display(result_df7)

,변수,상관계수
0,잔액_신판ca최대한도소진율_r6m,0.268486
1,잔액_신판ca최대한도소진율_r3m,0.263688
2,잔액_신판ca평균한도소진율_r3m,0.258636
3,잔액_신판ca평균한도소진율_r6m,0.252525
4,잔액_신판최대한도소진율_r6m,0.200087
5,잔액_신판최대한도소진율_r3m,0.194820
6,잔액_신판평균한도소진율_r3m,0.189758
7,잔액_신판평균한도소진율_r6m,0.181995


In [146]:
# 비교할 종속변수
target = 'Segment_e'
# 결과를 담을 리스트 
corr_result = []
# 반복
for col in column6:
    if col in all_df.columns:
        corr = all_df[col].corr(all_df[target])
        corr_result.append({'변수': col, '상관계수': corr})

# 데이터프레임으로 정리한다 
result_df8 = pd.DataFrame(corr_result).sort_values(by='상관계수', key=abs, ascending=False).reset_index(drop=True)

display(result_df8)

,변수,상관계수
0,변동률_RV일시불평잔,0.104848
1,변동률_RVCA평잔,0.104127
2,변동률_카드론평잔,0.072186
3,변동률_일시불평잔,0.057162
4,변동률_CA평잔,0.044449
5,변동률_할부평잔,0.020413


In [147]:
# 비교할 종속변수
target = 'Segment_e'
# 결과를 담을 리스트 
corr_result = []
# 반복
for col in column7:
    if col in all_df.columns:
        corr = all_df[col].corr(all_df[target])
        corr_result.append({'변수': col, '상관계수': corr})

# 데이터프레임으로 정리한다 
result_df9 = pd.DataFrame(corr_result).sort_values(by='상관계수', key=abs, ascending=False).reset_index(drop=True)

display(result_df9)

,변수,상관계수
0,혜택수혜율_B0M,-0.099772
1,혜택수혜율_R3M,-0.093787
2,변동률_잔액_CA_B1M,-0.013722
3,변동률_잔액_일시불_B1M,0.003956
4,변동률_잔액_B1M,0.003602


In [149]:
# 비교할 종속변수
target = 'Segment_e'
# 결과를 담을 리스트 
corr_result = []
# 반복
for col in column8:
    if col in all_df.columns:
        corr = all_df[col].corr(all_df[target])
        corr_result.append({'변수': col, '상관계수': corr})

# 데이터프레임으로 정리한다 
result_df10 = pd.DataFrame(corr_result).sort_values(by='상관계수', key=abs, ascending=False).reset_index(drop=True)

display(result_df10)

,변수,상관계수
0,증감율_이용건수_카드론_분기,-0.023503
1,증감율_이용금액_카드론_분기,-0.023284
2,증감율_이용금액_카드론_전월,-0.006121
3,증감율_이용건수_카드론_전월,-0.005304


In [151]:
corr_df = pd.concat(
    [result_df3, result_df4, result_df5, result_df6, result_df7, result_df8, result_df9, result_df10],
    axis=0,
    ignore_index=True
)

In [165]:
corr_result_df = corr_df.sort_values(by='상관계수', ascending=False).reset_index(drop=True)
corr_result_df

,변수,상관계수
0,잔액_신판ca최대한도소진율_r6m,0.268486
1,잔액_신판ca최대한도소진율_r3m,0.263688
2,잔액_신판ca평균한도소진율_r3m,0.258636
3,잔액_신판ca평균한도소진율_r6m,0.252525
4,잔액_신판최대한도소진율_r6m,0.200087
5,잔액_신판최대한도소진율_r3m,0.194820
6,잔액_신판평균한도소진율_r3m,0.189758
7,잔액_신판평균한도소진율_r6m,0.181995
8,변동률_RV일시불평잔,0.104848
9,변동률_RVCA평잔,0.104127


In [171]:
result_AK = pd.concat(
    [result_df1, result_df2],
    axis=0,
    ignore_index=True
)
display(result_AK)

,변수,유형,계수종류,상관계수
0,잔액_신판ca최대한도소진율_r6m,수치형,Eta²,0.075072
1,잔액_신판ca최대한도소진율_r3m,수치형,Eta²,0.072253
2,잔액_신판ca평균한도소진율_r3m,수치형,Eta²,0.069510
3,잔액_신판ca평균한도소진율_r6m,수치형,Eta²,0.066496
4,잔액_신판최대한도소진율_r6m,수치형,Eta²,0.044228
5,잔액_신판최대한도소진율_r3m,수치형,Eta²,0.042057
6,잔액_신판평균한도소진율_r3m,수치형,Eta²,0.040209
7,잔액_신판평균한도소진율_r6m,수치형,Eta²,0.037250
8,변동률_RV일시불평잔,수치형,Eta²,0.012765
9,변동률_RVCA평잔,수치형,Eta²,0.012526


In [173]:
### 내보낸다. 
result_AK.to_excel('성과정보_상관계수.xlsx', index=False)
corr_result_df.to_excel('성과정보_검사.xlsx', index=False)


### VIF

In [ ]:
# 사용할 수치형 변수 리스트
features = ['잔액_신판평균한도소진율_r6m', '잔액_신판최대한도소진율_r6m',
       '잔액_신판평균한도소진율_r3m', '잔액_신판최대한도소진율_r3m', '잔액_신판ca평균한도소진율_r6m',
       '잔액_신판ca최대한도소진율_r6m', '잔액_신판ca평균한도소진율_r3m', '잔액_신판ca최대한도소진율_r3m', '변동률_RV일시불평잔'
        ,'변동률_RVCA평잔']

features1 = ['잔액_신판ca최대한도소진율_r6m', '변동률_RVCA평잔']

# 독립변수 데이터
X = all_df[features]
X = add_constant(X)  # 상수항 추가

# VIF 계산
vif_df = pd.DataFrame()
vif_df['Feature'] = X.columns
vif_df['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_df)

In [ ]:
all_df[column5].corr()

# ANOVA

In [ ]:
from scipy.stats import f_oneway

segment_col = 'Segment'
results = []

for col in numerical_cols:  # 수치형 변수 리스트
    try:
        groups = [all_df[all_df[segment_col] == grp][col].dropna() for grp in all_df[segment_col].unique()]
        if len(groups) < 2 or any(len(g) < 2 for g in groups):
            continue  # 그룹 수가 부족하면 skip

        f_stat, p_value = f_oneway(*groups)

        results.append({
            '변수': col,
            'F-stat': round(f_stat, 4),
            'p-value': round(p_value, 6),
            '유의함': '✅' if p_value < 0.05 else ''
        })
    except:
        continue

anova_df = pd.DataFrame(results).sort_values(by='p-value')
display(anova_df)


### ANOVA & Turkey

목적은 Segment라는 범주형 타깃을 예측하는 것.
즉, 모델은 입력 변수(X)를 보고 A, B, C, D, E 중 어디에 속하는지를 분류해야함

In [ ]:

# 1. ANOVA: Segment에 따라 평균차이 있는지
def run_anova(df, target_col, feature_col):
    groups = [df[df[target_col] == g][feature_col].dropna() for g in df[target_col].unique()]
    return f_oneway(*groups)

# 2. Tukey HSD: 그룹 간 차이 상세 분석
def run_tukey(df, target_col, feature_col):
    tukey = pairwise_tukeyhsd(df[feature_col], df[target_col], alpha=0.05)
    return pd.DataFrame(tukey.summary().data[1:], columns=tukey.summary().data[0])

# 3. 시각화
def plot_box(df, group_col, target_col):
    plt.figure(figsize=(8, 5))
    sns.boxplot(x=group_col, y=target_col, data=df)
    plt.title(f'Boxplot of {target_col} by {group_col}')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
numerical_cols = [col for col in column1 + column2 if col != 'Segment_e']
anova_results = []
tukey_all_results = {}

for col in numerical_cols:
    try:
        fstat, pval = run_anova(all_df, 'Segment', col)
        conclusion = '✅ 유의함' if pval < 0.05 else '❌ 불충분'
        anova_results.append({'변수': col, 'F-stat': round(fstat, 4), 'p-value': round(pval, 6), '결론': conclusion})
        
        if pval < 0.05:
            print(f"\n📌 유의한 변수: {col} → Tukey HSD & Boxplot 진행 중...")
            tukey_df = run_tukey(all_df, 'Segment', col)
            tukey_all_results[col] = tukey_df  # 저장
            display(tukey_df)
            plot_box(all_df, 'Segment', col)

    except Exception as e:
        anova_results.append({'변수': col, 'F-stat': None, 'p-value': None, '결론': f'⚠ 오류: {e}'})
